<a href="https://colab.research.google.com/github/juanigp/IterNet/blob/master/Testeo_escena_5000x3500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para segmentar la escena de 5000x3500

In [1]:
!git clone https://github.com/juanigp/IterNet
%cd /content/IterNet

Cloning into 'IterNet'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 177 (delta 7), reused 15 (delta 4), pack-reused 153
Receiving objects: 100% (177/177), 109.12 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (84/84), done.
/content/IterNet


Colab se cierra si la imagen de entrada es muy grande, o si se quiere testear muchas imagenes a la vez. Lo siguiente es para extraer recortes de 500x500, con stride 250x250, de a 20 por vez, testearlos, borrar esa data y continuar con los proximos 20. El algoritmo tiene algun error porque genera algunas imagenes duplicadas.

In [0]:
from PIL import Image
from glob import glob
import re
import pickle
import predict
import numpy as np
scene_dir = r'raw data/escena 2/escena 2_5000x3500.bmp'
#mask_dir = r'raw data/escena 2/escena 2_5000x3500.gif'
scene = np.array( Image.open(scene_dir) )
#mask = np.array( Image.open(mask_dir) )
v_x, v_y = 500, 500
s_x, s_y = 250, 250
count = 0
big_batch = 20
for i in range(v_x - 1, scene.shape[0] + s_x, s_x):
  for j in range (v_y - 1, scene.shape[1] + s_y, s_y):
    #crear los 20 recortes y guardarlos en el directorio de entrada para la red
    i_max = min(i, scene.shape[0] - 1)
    j_max = min(j, scene.shape[1] - 1)
    i_min, j_min = i_max -(v_x - 1), j_max -(v_y - 1)
    patch_scene =  np.zeros([v_x, v_y, 3])
    #patch_mask =  np.zeros([v_x, v_y])
    patch_scene[0 : v_x, 0 : v_y] = scene[i_min:(i_max+1), j_min:(j_max+1)]
    #patch_mask[0 : v_x, 0 : v_y] = mask[i_min:(i_max+1), j_min:(j_max+1)]
    patch_scene = patch_scene.astype(np.uint8)
    patch_scene_img = Image.fromarray(patch_scene)
    #patch_mask_img = Image.fromarray(patch_mask)
    count += 1   
    patch_scene_img.save(r'data/DRIVE/test/images/'+format(count, '05d')+'.bmp')
    #patch_mask_img.save(r'data/DRIVE/test/1st_manual/'+format(count, '05d')+'.gif')
    #cuando se tienen 20:
    if (count % big_batch == 0):
      #prediccion
      predict.predict(stride_size=10)
      #todos los .pickle generados
      files = glob(r"output/DRIVE/crop_size_128/out4/*.pickle")
      for f in files:
        #se guarda cada pickle en el directorio results, y tambien se exporta como imagen
        #es importante que que los resultados queden registrados con el nombre correcto para poder comparar!!!
        pkl = pickle.load(open( f, "rb" ) )
        im = Image.fromarray(pkl*255)
        im = im.convert("L")
        result = re.search('out4/(.*).pickle', f)
        name = str( int( result.group(1) ) + count - big_batch )
        im.save(r'results/' + name + '.png')
        pickle.dump( pkl, open( r'results/' + name + '.pickle', "wb" ) )
      #se copian los recortes originales y sus mascaras, y luego se eliminan para dejar lugar a los proximos 20 recortes
      %cp -r data/DRIVE/test/images/* results
      %cp -r data/DRIVE/test/1st_manual/* results
      %rm data/DRIVE/test/images/*
      %rm data/DRIVE/test/1st_manual/*
      %rm -rf data/HDF5
      %rm -rf output

#si quedaron recortes sin analizar:
if (count % big_batch != 0):
  predict.predict(stride_size=10)
  files = glob(r"output/DRIVE/crop_size_128/out4/*.pickle")
  for f in files:
    pkl = pickle.load(open( f, "rb" ) )
    im = Image.fromarray(pkl*255)
    im = im.convert("L")
    result = re.search('out4/(.*).pickle', f)
    name = str( int( result.group(1) ) + count - (count % big_batch) )
    im.save(r'results/' + name + '.png')
  %cp -r data/DRIVE/test/images/* results
  %cp -r data/DRIVE/test/1st_manual/* results
  %rm data/DRIVE/test/images/*
  %rm data/DRIVE/test/1st_manual/*
  %rm -rf output
    

In [0]:
#Copiar todo a un Google Drive
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

%mkdir "/gdrive/My Drive/big_scene_results" #o el directorio que se quiera
%cp -r results "/gdrive/My Drive/big_scene_results"
